In this notebook we learn about how to use interpretable machine learning for a Regression model n California housing dataset. 
Here we study about both Global and Local explanations to understand the models overall prediction as well as a single 
prediction of an outcome. We also compare the performance of a Regression tree.

## Setup a regression experiment

In [1]:
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

california_housing = fetch_california_housing()
feature_names = list(california_housing.feature_names)
df = pd.DataFrame(california_housing.data, columns=feature_names)
df["target"] = california_housing.target
# df = df.sample(frac=0.1, random_state=1)
train_cols = df.columns[0:-1]
label = df.columns[-1]
X = df[train_cols]
y = df[label]

seed = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

## Explore the dataset

In [2]:
from interpret import show
from interpret.data import Marginal

marginal = Marginal().explain_data(X_train, y_train, name = 'Train Data')
show(marginal)

/opt/anaconda3/lib/python3.8/site-packages/interpret/visual/udash.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/opt/anaconda3/lib/python3.8/site-packages/interpret/visual/udash.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/opt/anaconda3/lib/python3.8/site-packages/interpret/visual/udash.py:7: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table as dt


<!-- http://127.0.0.1:7755/140426156418672/ -->

## Train the Explainable Boosting Machine (EBM)

In [11]:
from interpret.glassbox import ExplainableBoostingRegressor, LinearRegression, RegressionTree

ebm = ExplainableBoostingRegressor(random_state=seed, n_jobs=-1)
ebm.fit(X_train, y_train)   #Works on dataframes and numpy arrays

ExplainableBoostingRegressor(feature_names=['MedInc', 'HouseAge', 'AveRooms',
                                            'AveBedrms', 'Population',
                                            'AveOccup', 'Latitude', 'Longitude',
                                            'HouseAge x AveOccup',
                                            'Latitude x Longitude',
                                            'MedInc x AveOccup',
                                            'AveOccup x Latitude',
                                            'MedInc x Longitude',
                                            'AveBedrms x Longitude',
                                            'MedInc x HouseAge',
                                            'AveOccup x Longitude',
                                            'HouseAge x Longitude',
                                            'Population x AveOccup'],
                             feature_types=['continuous', 'continuous',
                                            'continuous', 'continuous',
                                            'continuous', 'continuous',
                                            'continuous', 'continuous',
                                            'interaction', 'interaction',
                                            'interaction', 'interaction',
                                            'interaction', 'interaction',
                                            'interaction', 'interaction',
                                            'interaction', 'interaction'],
                             n_jobs=-1, random_state=1)

## Global Explanations: What the model learned overall

In [4]:
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)

<!-- http://127.0.0.1:7755/140426225983152/ -->

## Local Explanations: How an individual prediction was made

In [5]:
ebm_local = ebm.explain_local(X_test[:5], y_test[:5], name='EBM')
show(ebm_local)

<!-- http://127.0.0.1:7755/140426226135920/ -->

## Evaluate EBM performance

In [6]:
from interpret import show
from interpret.perf import RegressionPerf

ebm_perf = RegressionPerf(ebm.predict).explain_perf(X_test, y_test, name='EBM')
show(ebm_perf)

<!-- http://127.0.0.1:7755/140426166640016/ -->

## Let's test out a few other Explainable Models

In [7]:
from interpret.glassbox import LinearRegression, RegressionTree

lr = LinearRegression(random_state=seed)
lr.fit(X_train, y_train)

rt = RegressionTree(random_state=seed)
rt.fit(X_train, y_train)

## Compare performance using the Dashboard

In [8]:
lr_perf = RegressionPerf(lr.predict).explain_perf(X_test, y_test, name='Linear Regression')
rt_perf = RegressionPerf(rt.predict).explain_perf(X_test, y_test, name='Regression Tree')

show(lr_perf)
show(rt_perf)
show(ebm_perf)

<!-- http://127.0.0.1:7755/140426166637952/ -->

<!-- http://127.0.0.1:7755/140426166637088/ -->

<!-- http://127.0.0.1:7755/140426166640016/ -->

## Glassbox: All of our models have global and local explanations

In [9]:
lr_global = lr.explain_global(name='Linear Regression')
rt_global = rt.explain_global(name='Regression Tree')

show(lr_global)
show(rt_global)
show(ebm_global)

<!-- http://127.0.0.1:7755/140426191536624/ -->

<!-- http://127.0.0.1:7755/140426198883152/ -->

<!-- http://127.0.0.1:7755/140426225983152/ -->

## Dashboard: look at everything at once

In [10]:
# Do everything in one shot with the InterpretML Dashboard by passing a list into show

show([marginal, lr_global, lr_perf, rt_global, rt_perf, ebm_global, ebm_perf])

<!-- http://127.0.0.1:7755/140426199127680/ -->
 Open in new window